In [1]:
import openai
import pandas as pd
import json
from IPython.display import Markdown
import sys

In [2]:
zz=openai.Model.list()

In [3]:
len(zz['data'])

62

In [4]:
[zz['data'][idx]['id'] for idx in range(len(zz['data'])) if 'gpt' in zz['data'][idx]['id']]

['gpt-4-0613',
 'gpt-4-0314',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo',
 'gpt-4']

In [5]:
model_params = {
    'model': "gpt-4",
    'temperature': 0,
}

In [10]:
def print_save(string, file='../cmp2.md'):
    print(string)
    with open(file,'w') as f:
        f.write(string)

## Load `prompt_template.md`

In [2]:
sys.path.append('..')
from utils import *
from functools import partial

In [3]:
# prompt_dict=load_prompt_template('../prompt_template.md')
prompt_dict = partial(load_prompt_template,file='../prompt_template.md')

In [9]:
prompt_dict().keys()

dict_keys(['Preamble', 'Construct Kinetic Hamiltonian (continuum version)', 'Construct Kinetic Hamiltonian (lattice version)', 'Define each term in Kinetic Hamiltonian (continuum version)', 'Construct Potential Hamiltonian (continuum version)', 'Define each term in Potential Hamiltonian (continuum version)', 'Construct interaction Hamiltonian (real space, lattice version)', 'Construct interaction Hamiltonian (momentum space)', 'Convert from single-particle to second-quantized form, return in matrix', 'Convert from single-particle to second-quantized form, return in summation (expand the matrix)', 'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)', 'Convert noninteracting Hamiltonian in real space to momentum space (lattice version)', 'Convert interacting Hamiltonian in real space to momentum space (lattice version)', 'Particle-hole transformation', 'Simplify the Hamiltonian in the particle-hole basis', "Wick's theorem", 'Extract quadratic term', 'H

In [4]:
with open('2108.02159.jsonl','r') as f:
    kwargs= [json.loads(line) for line in f]


In [5]:
import re
import string
def format_check(s, kwargs):
    placeholders=  [v[1] for v in (string.Formatter().parse(s)) if v[1] is not None]
    extra_args = set(kwargs.keys()) - set(placeholders+ ['task','answer'])  # Check for extra args.
    return extra_args


In [7]:
for kwarg in kwargs:
    set_diff=format_check(prompt_dict[kwarg['task']],kwarg)
    if len(set_diff)>0:
        print(kwarg['task'],set_diff)

Expand interaction {'def_var'}


In [6]:
[format_check(prompt_dict[kwarg['task']],kwarg) for kwarg in kwargs]

[set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 {'def_var'},
 set(),
 set(),
 set()]

In [11]:
df=pd.DataFrame(list(prompt_dict().items()),columns=['Task','Prompt'])

## Construct Kinetic Hamiltonian (continuum version)

In [11]:
user_msg=[]
AI_msg=[]
sys_msg=[{'role': 'system', 'content': prompt_dict()['Preamble']}]
kwargs_list=[]

In [12]:
kwargs = {
    'task': 'Construct Kinetic Hamiltonian (continuum version, second-quantized)',
    'system': 'moire TMD',
    'real|momentum': 'momentum',
    'single-particle|second-quantized': 'second-quantized',
    'dof': 'moire reciprocal lattice vectors $b$ (infinite set), spin index (spin up and spin down)',
    'kinetic_symbol': r'$H_{Kinetic}(k)$',
    'dispersion_symbol': r'$E_{\sigma,b}(k)$',
    'annihilation_op':r'$c_{\sigma,b}(k)$',
    'creation_op':r'$c_{\sigma,b}^\dagger(k)$',
    'k|r': 'k',
    'entire_space|first_Brillouin_zone': r'the first Brillouin zone (BZ)',
    'def_var': r'''$b$: moire reciprocal lattice vectors
$k$: momentum within the first Brillouin zone
$\sigma$: spin index  
$\uparrow$: spin up  
$\downarrow$: spin down  
$c_{\sigma,b}(k)$ : annihilation operator with spin $\sigma$ and moire reciprocal lattice vectors $b$  
$c_{\sigma,b}^\dagger(k)$ : creation operator with spin $\sigma$ and moire reciprocal lattice vectors $b$   
$E_{\sigma,b}(k)$: energy dispersion at spin $\sigma$ and with moire reciprocal lattice vectors $b$.  
$H_{Kinetic}$: the total Kinetic energy in the second quantized form''',
    'answer':    r'$H_{Kinetic} = \sum_{\sigma,b,k} E_{\sigma,b}(k) c_{\sigma,b}^\dagger(k) c_{\sigma,b}(k)$',
'source': {'2108.02159.tex':[(168, 279)]}

}


In [13]:
# text=''
# for file, position in kwargs['source'].items():
#     with open(file,'r') as f:
#         f_list=list(f)
#         for left,right in position:
#             text+=''.join(f_list[left-1:right])
# print(text)

In [14]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to describe the kinetic term of Hamiltonian in moire TMD in the momentum space in the second-quantized form.   
The degrees of freedom of the system are: moire reciprocal lattice vectors $b$ (infinite set), spin index (spin up and spin down).  
Express the Kinetic Hamiltonian $H_{Kinetic}(k)$ using $E_{\sigma,b}(k)$, $c_{\sigma,b}(k)$, and $c_{\sigma,b}^\dagger(k)$, where the summation of $k$ should be running over the the first Brillouin zone (BZ).

Use the following conventions for the symbols:  
$b$: moire reciprocal lattice vectors
$k$: momentum within the first Brillouin zone
$\sigma$: spin index  
$\uparrow$: spin up  
$\downarrow$: spin down  
$c_{\sigma,b}(k)$ : annihilation operator with spin $\sigma$ and moire reciprocal lattice vectors $b$  
$c_{\sigma,b}^\dagger(k)$ : creation operator with spin $\sigma$ and moire reciprocal lattice vectors $b$   
$E_{\sigma,b}(k)$: energy dispersion at spin $\sigma$ and with moire reciprocal lattice vectors $b$.  
$H

In [15]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [16]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [17]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [18]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [19]:
# rs['choices'][0]['message'].content

In [20]:
kwargs_list.append(kwargs)

## Define each term in Kinetic Hamiltonian (continuum version)

In [21]:
# user_msg=[]
# AI_msg=[]
# sys_msg=[{'role': 'system', 'content': prompt_dict()['Preamble']}]
# kwargs_list=[]

In [22]:
kwargs = {
    'task': 'Define each term in Kinetic Hamiltonian (continuum version)',
    'var':r'the energy dispersion $E_{\sigma,b}(k)$',
    'Description':r'''For all energy dispersions, $E_{\sigma,b}(k)$, it characterizes the dispersion for free holes, namely, $E_{\sigma,b}(k)= -\frac{\hbar^2 (k+b)^2}{2 m^*}$.   ''',
    'expression_kinetic': r'$H_{Kinetic} = \sum_{\sigma,b,k} E_{\sigma,b}(k) c_{\sigma,b}^\dagger(k) c_{\sigma,b}(k)$',
    'kinetic_symbol': r'$H_{Kinetic}(k)$',
    'def_var': r'''$m^*$ : effective mass  ''',
'answer':r"$H_{Kinetic} = -\frac{\hbar^2}{2 m^*} \sum_{\sigma,b,k} (k+b)^2 c_{\sigma,b}^\dagger(k) c_{\sigma,b}(k)$",
'source': {'2108.02159.tex':[(168, 279)]}

}

In [23]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct each term, namely the energy dispersion $E_{\sigma,b}(k)$.  
For all energy dispersions, $E_{\sigma,b}(k)$, it characterizes the dispersion for free holes, namely, $E_{\sigma,b}(k)= -\frac{\hbar^2 (k+b)^2}{2 m^*}$.   .  
You should recall that $H_{Kinetic} = \sum_{\sigma,b,k} E_{\sigma,b}(k) c_{\sigma,b}^\dagger(k) c_{\sigma,b}(k)$.   
Return the expression for the energy dispersion $E_{\sigma,b}(k)$ in the Kinetic Hamiltonian, and substitute it into the Kinetic Hamiltonian $H_{Kinetic}(k)$.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$m^*$ : effective mass


In [24]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [25]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [26]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)
# Markdown(rs['choices'][0]['message'].content)

In [27]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [28]:
# rs['choices'][0]['message'].content

In [29]:
kwargs_list.append(kwargs)

## Construct Potential Hamiltonian (continuum version)

In [30]:
# user_msg=[]
# AI_msg=[]
# sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]
# kwargs_list=[]

In [31]:
kwargs = {
    'task':'Construct Potential Hamiltonian (continuum version)',
    'potential_symbol':r'$H_{Potential}(k)$',
    'real|momentum':'momentum',
    'single-particle|second-quantized':'second-quantized',
    'kinetic_symbol': r'$H_{Kinetic}(k)$',
    'Description': '',
    'var':
    r"$\Delta_{\sigma, b,b'}$ and $c_{\sigma,b}(k)$, and $c_{\sigma,b'}^\dagger(k)$",

    'def_var': r'''$H_{Potential}$: the potential term in the second quantized form''',
    'answer':r"$H_{Potential} = \sum_{\sigma,b,b',k} \Delta_{\sigma, b,b'} c_{\sigma,b}^\dagger(k) c_{\sigma,b'}(k)$",
'source': {'2108.02159.tex':[(168, 279)]}

}


In [32]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to describe the potential term of Hamiltonian $H_{Potential}(k)$ in the momentum space in the second-quantized form.  
The potential Hamiltonian has the same degrees of freedom as the kinetic Hamiltonian $H_{Kinetic}(k)$.  
.  
Express the potential Hamiltonian $H_{Potential}(k)$ using $\Delta_{\sigma, b,b'}$ and $c_{\sigma,b}(k)$, and $c_{\sigma,b'}^\dagger(k)$.  

Use the following conventions for the symbols (You should also remember the conventions in my previous prompts if there are no conflicts. If you have conflicts in the conventions, you should stop and let me know):  
$H_{Potential}$: the potential term in the second quantized form


In [33]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [34]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [35]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [36]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [37]:
# rs['choices'][0]['message'].content

In [38]:
kwargs_list.append(kwargs)

## Define each term in Potential Hamiltonian (continuum version)

In [39]:
# user_msg=[]
# AI_msg=[]
# sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]
# kwargs_list=[]

In [40]:
kwargs = {
    'task':
    'Define each term in Potential Hamiltonian (continuum version)',
    'potential_symbol':'$H_{Potential}(k)$',
    'var':    r"$\Delta_{\sigma, b,b'}$",
    'Description': r'''$\Delta_{\sigma, b,b'}=\sum_{j=1}^6 V_j \delta_{b_j,b-b'}$, where $V_j=V_Me^{(-1)^{j-1}i\phi}$''',
    'def_var': r'''$b_j$: first shell of moire reciprocal lattice vectors  
$V_M$: characterizes moire modulation strength  
$\phi$: phase factor defines the moire modulation phase  
$\delta_{i,j}$ : Kronecker delta function, take the value of 1 if $i=j$, and 0 otherwise.''',
    'expression_Potential': r"H_{Potential} = \sum_{\sigma,b,b',k} \Delta_{\sigma, b,b'} c_{\sigma,b}^\dagger(k) c_{\sigma,b'}(k)",
    'answer':r"$H_{Potential} = \sum_{\sigma,b,b',k, j=1}^6 V_j \delta_{b_j,b-b'} c_{\sigma,b}^\dagger(k) c_{\sigma,b'}(k)$",
'source': {'2108.02159.tex':[(168, 279)]}

}


In [41]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct each term $H_{Potential}(k)$, namely, $\Delta_{\sigma, b,b'}$.  
$\Delta_{\sigma, b,b'}=\sum_{j=1}^6 V_j \delta_{b_j,b-b'}$, where $V_j=V_Me^{(-1)^{j-1}i\phi}$.
You should recall that H_{Potential} = \sum_{\sigma,b,b',k} \Delta_{\sigma, b,b'} c_{\sigma,b}^\dagger(k) c_{\sigma,b'}(k).  
Return the expressions for $\Delta_{\sigma, b,b'}$, and substitute it into the potential Hamiltonian $H_{Potential}(k)$.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or have conflicts in the conventions, you should stop and let me know):  
$b_j$: first shell of moire reciprocal lattice vectors  
$V_M$: characterizes moire modulation strength  
$\phi$: phase factor defines the moire modulation phase  
$\delta_{i,j}$ : Kronecker delta function, take the value of 1 if $i=j$, and 0 otherwise.


In [42]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [43]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [44]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [45]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [46]:
# rs['choices'][0]['message'].content

In [47]:
kwargs_list.append(kwargs)

## Construct interaction Hamiltonian (momentum space)

In [48]:
kwargs = {
    'task':'Construct interaction Hamiltonian (momentum space)',
    'second_int_symbol':  r'$\hat{H}^{int}$',
    'index': r'spin index $\sigma_i$',
    'momentum': r'momentum $k_i$ and moire reciprocal lattice vectors $b_i$',
    'For each operator, the total momentum is the sum of moire reciprocal lattice $b_i$ and momentum with in the first BZ $k_i$|None': r'For each operator, the total momentum is the sum of moire reciprocal lattice $b_i$ and momentum with in the first BZ $k_i$',
    'interaction': 'bare Coulomb interaction',
    'int_form': r'2\pi e^2/(\epsilon |q|)',
    'normalization_factor': r'$\frac{1}{2 N V}$',
    'op': r'$c_{\sigma,b}(k)$',
    'def_var': r'''$\epsilon$ : dielectric constant  
$V(q)$ : bare Coulomb interaction in the momentum space''',
    'answer':r'$\hat{H}^{int} = \frac{1}{2 N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) c_{\sigma_1,b_4}(k_4) V(q) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4}$',
'source': {'2108.02159.tex':[(281, 291)]}

}


In [49]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct the interaction part of the Hamiltonian $\hat{H}^{int}$ in the momentum space.  
The interaction Hamiltonian is a product of four parts.
The first part is the product of four operators with two creation and two annihilation operators following the normal order, namely, creation operators are before annihilation operators. You should follow the order of $1,2,2,1$ for the spin index $\sigma_i$, and $1,2,3,4$ for the momentum $k_i$ and moire reciprocal lattice vectors $b_i$. 
The second part is the constraint of total momentum conservation, namely the total momentum of all creation operators should be the same as that of all annihilation operators. For each operator, the total momentum is the sum of moire reciprocal lattice $b_i$ and momentum with in the first BZ $k_i$  
The third part is the interaction form. You should use bare Coulomb interaction with $V(q)=2\pi e^2/(\epsilon |q|)$, where $q$ is the transferred total momentum between a creation opera

In [50]:
kwargs_list.append(kwargs)

## Wick's theorem

In [51]:
kwargs = {
    'task':"Wick's theorem",
    'second_int_symbol':  r'$\hat{H}^{int}$',
    'HF_symbol': r'$\hat{H}^{int,HF}$',
    'expression_int': r'$\hat{H}^{int} = \frac{1}{2 N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) c_{\sigma_1,b_4}(k_4) V(q) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4}$',
    'def_var': r'''$\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_2}(k_2) \rangle$ : expectation of $c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_2}(k_2)$ over the many-body ground states  
$\hat{H}^{int,HF}$ : Interaction term after Hartree-Fock approximation''',
    'answer':r'$\hat{H}^{int,HF} = \frac{1}{2 N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} V(q) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4} [\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3)+ \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4)- \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle- \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4)- \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3)+ \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) \rangle]$',
'source': {'2108.02159.tex':[(447, 480)]}

}


In [52]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to perform a Hartree-Fock approximation to expand the interaction term, $\hat{H}^{int}$.  
You should use Wick's theorem to expand the four-fermion term in $\hat{H}^{int}$ into quadratic terms. You should strictly follow the EXAMPLE below to expand using Wick's theorem, select the correct EXAMPLE by noticing the order of four term product with and without ${}^\dagger$, and be extremely cautious about the order of the index and sign before each term.  
You should only preserve the normal terms. Here, the normal terms mean the product of a creation operator and an annihilation operator.  
You should recall that $\hat{H}^{int} = \frac{1}{2 N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) c_{\sigma_1,b_4}(k_4) V(q) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4}$.  
Return the expanded interaction term after Hartree-Fock approximation as $\hat{H}^{int,HF}$.

Use the follo

In [53]:
kwargs_list.append(kwargs)

## Extract quadratic term

In [54]:
kwargs = {
    'task':"Extract quadratic term",
    'HF_symbol': r'$\hat{H}^{int,HF}$',
    'bilinear_op': r'$c^\dagger c$',
    'HF_2_symbol': r'$\hat{H}^{int,HF,2}$',
    'expression_HF': r'$\hat{H}^{int,HF} = \frac{1}{2 N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} V(q) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4} [\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3)+ \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4)- \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle- \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4)- \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3)+ \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) \rangle]$',
    'def_var': r'''$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$''',
    'answer':r'\hat{H}^{int,HF,2} = \frac{1}{2 N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} V(q) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4} [\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) + \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) - \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) - \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3)]',
'source': {'2108.02159.tex':[(447, 480)]}

}


In [55]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to extract the quadratic terms in the $\hat{H}^{int,HF}$.  
The quadratic terms mean terms that are proportional to $c^\dagger c$, which excludes terms that are solely expectations or products of expectations.  
You should only preserve the quadratic terms in $\hat{H}^{int,HF}$, denoted as $\hat{H}^{int,HF,2}$.  
You should recall that $\hat{H}^{int,HF} = \frac{1}{2 N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} V(q) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4} [\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3)+ \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4)- \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle- \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b

In [56]:
kwargs_list.append(kwargs)

## Expand interaction

In [57]:
kwargs = {
    'task':"Expand interaction",
    'HF_2_symbol': r'$\hat{H}^{int,HF,2}$',
    'momentum': r'$b_i$ and $k_i$',
    'expression_HF_2': r'$\hat{H}^{int,HF,2} = \frac{1}{2 N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} V(q) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4} [\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) + \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) - \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) - \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3)]$',
    'answer':r'\hat{H}^{int,HF,2} = \frac{1}{2 N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} V(k_1+b_1-k_4-b_4) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4} [\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) + \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) - \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) - \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3)]',
'source': {'2108.02159.tex':[(447, 480)]}

}


In [58]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to expand interaction term $V(q)$ in the MF quadratic term $\hat{H}^{int,HF,2}$.
If you find the $V(q)$ in $\hat{H}^{int,HF,2}$ does not contain any momentum that is not in the summation sign. The interaction term is already expanded. No action to perform on interaction term.
Otherwise, you will expand $V(q)$ by replacing $q$ with the momentum $b_i$ and $k_i$.
You should recall that $\hat{H}^{int,HF,2} = \frac{1}{2 N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} V(q) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4} [\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) + \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) - \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) - \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) \rangle c_{\si

In [59]:
kwargs_list.append(kwargs)

## Swap the index to combine Hartree and Fock terms

In [60]:
kwargs = {
    'task':"Swap the index to combine Hartree and Fock terms",
    'HF_2_symbol': r'$\hat{H}^{int,HF,2}$',
    'expval': r'$\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle$',
    'expression_Hartree_1':r'$\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3)$',
    'expression_Hartree_2':r'$\langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4)$',
    'expression_HF_2':r'$\hat{H}^{int,HF,2} = \frac{1}{2 N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} V(k_1+b_1-k_4-b_4) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4} \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) + \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) - \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) - \langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3)$',
    'answer':r'$\hat{H}^{int,HF,2} = \frac{1}{N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} V(k_1+b_1-k_4-b_4) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4} [\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) - \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4)]$',
'source': {'2108.02159.tex':[(447, 480)]}

}


In [61]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the quadratic term $\hat{H}^{int,HF,2}$ through relabeling the index to combine the two Hartree/Fock term into one Hartree/Fock term.  
The logic is that the expected value ($\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle$) in the first Hartree term ($\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3)$) has the same form as the quadratic operators in the second Hartree term ($\langle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4)$), and vice versa. The same applies to the Fock term.  
This means, if you relabel the index by swapping the index in the "expected value" and "quadratic operators" in the second Hartree term, you can make the second Hartree term look identical to the first Hartree term, as long as $V(q)=V(-q)$, which is naturally satisfied in Coulomb interaction. You should follow the EXAM

In [62]:
kwargs_list.append(kwargs)

## Reduce momentum in Hartree term (momentum in BZ + reciprocal lattice)

In [63]:
kwargs = {
    'task':"Reduce momentum in Hartree term (momentum in BZ + reciprocal lattice)",
    'expval': r'$\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle$',
    'expval_id':r'$\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle=\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle \delta_{k_i,k_j}$',

    'expression_Hartree':r'$\hat{H}^{int,Hartree,2}= \frac{1}{N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} V(k_1+b_1-k_4-b_4) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4}\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_4) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_3)$',
    'Hartree_2_symbol':r'$\hat{H}^{int,Hartree,2}$',
    'answer':r'$\hat{H}^{int,Hartree,2}= \frac{1}{N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2} V(b_1-b_4) \delta_{b_1+b_2,b_3+b_4}\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_1,b_4}(k_1) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_2,b_3}(k_2)$',
'source': {'2108.02159.tex':[(447, 480)]}

}


In [64]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the Hartree term in $\hat{H}^{int,Hartree,2}$ by reducing the momentum inside the expected value $\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle$.  
The expected value $\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle$ is only nonzero when the two momenta $k_i,k_j$ are the same, namely, $\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle=\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle \delta_{k_i,k_j}$.  
You should use the property of Kronecker delta function $\delta_{k_i,k_j}$ to reduce one momentum $k_i$ but not $b_i$.
Once you reduce one momentum inside the expected value $\langle\dots\rangle$. You will also notice the total momentum conservation will reduce another momentum in the quadratic term. Therefore, you should end up with only two momenta left in the summation.  
You should follow the EXAMPLE below to reduce one momentum in the Hartree term, and another momentum 

In [65]:
kwargs_list.append(kwargs)

## Reduce momentum in Fock term (momentum in BZ + reciprocal lattice)

In [66]:
kwargs = {
    'task':"Reduce momentum in Fock term (momentum in BZ + reciprocal lattice)",
    'expval': r'$\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle$',
    'expval_id':r'$\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle=\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle \delta_{k_i,k_j}$',

    'expression_Fock':r'$\hat{H}^{int,Fock,2}= -\frac{1}{N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2, k_3, k_4} V(k_1+b_1-k_4-b_4) \delta_{k_1+b_1+k_2+b_2,k_3+b_3+k_4+b_4} \langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_3) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_4)$',
    'Fock_2_symbol':r'$\hat{H}^{int,Fock,2}$',
    'answer':r'$\hat{H}^{int,Fock,2}= -\frac{1}{N A} \sum_{\sigma_1, \sigma_2, b_1, b_2, b_3, b_4, k_1, k_2} V(k_1+b_1-k_2-b_4) \delta_{b_1+b_2,b_3+b_4}\langle c_{\sigma_1,b_1}^\dagger(k_1) c_{\sigma_2,b_3}(k_1) \rangle c_{\sigma_2,b_2}^\dagger(k_2) c_{\sigma_1,b_4}(k_2)$',
'source': {'2108.02159.tex':[(447, 480)]}
}


In [67]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the Fock term in $\hat{H}^{int,Fock,2}$ by reducing the momentum inside the expected value $\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle$.  
The expected value $\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle$ is only nonzero when the two momenta $k_i,k_j$ are the same, namely, $\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle=\langle c_{\sigma_i,b_i}^\dagger(k_i) c_{\sigma_j,b_j}(k_j) \rangle \delta_{k_i,k_j}$.  
You should use the property of Kronecker delta function $\delta_{k_i,k_j}$ to reduce one momentum $k_i$ but not $b_i$.  
Once you reduce one momentum inside the expected value $\langle\dots\rangle$. You will also notice the total momentum conservation will reduce another momentum in the quadratic term. Therefore, you should end up with only two momenta left in the summation.
You should follow the EXAMPLE below to reduce one momentum in the Fock term, and another momentum in the qu

In [68]:
kwargs_list.append(kwargs)

In [69]:
len(kwargs_list)

11

In [70]:
--

SyntaxError: invalid syntax (3659366440.py, line 1)

## Save jsonl

In [71]:
with open('2108.02159.jsonl', 'w') as f:
    for dict_obj in kwargs_list:
        f.write(json.dumps(dict_obj) + '\n')

# Draft